In [2]:
from sklearn.linear_model import LogisticRegression
import utils as utils
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [3]:
DATA_FILE_PATH = 'data.json'

In [4]:
train_tups, test_tups = utils.split_data(utils.generate_tuples_from_file(DATA_FILE_PATH, num_samples=10000), test_size=0.2)

print('Training set size: ', len(train_tups[0]))
print('Test set size: ', len(test_tups[0]))
print('Sample tokenized review: ', train_tups[0][0])
print('Sample rating: ', train_tups[1][0])

Training set size:  8000
Test set size:  2000
Sample tokenized review:  ['I', 'have', 'used', '``', 'vrbo', "''", 'and', 'home', 'away', 'websites', 'for', 'a', 'longtime', '.', 'This', 'is', 'the', 'worst', 'experience', 'I', 'have', 'ever', 'had', 'with', 'a', 'site', '.', 'My', 'friends', 'and', 'I', 'were', 'trying', 'to', 'book', 'a', 'place', 'for', 'the', 'Essence', 'Festival', 'in', 'July', '.', 'We', 'knew', 'how', 'the', 'housing', 'would', 'be', 'an', 'issue', 'so', 'we', 'started', 'looking', 'for', 'a', 'place', 'to', 'stay', 'at', 'the', 'end', 'of', 'Feb.', 'We', 'saw', 'the', 'French', 'Quarter', 'Suites', 'offered', 'Town', 'homes', ',', 'they', 'looked', 'nice', ',', 'so', 'we', 'began', 'to', 'inquire', '.', 'The', 'process', 'is', 'usually', 'pretty', 'easy', 'but', 'working', 'with', 'Ms.', 'Leslie', 'was', 'not', '.', 'When', 'asking', 'questions', 'about', 'a', '5br', 'town', 'home', 'she', 'would', 'often', 'skirt', 'the', 'questions', 'and', 'not', 'give', 'dir

In [11]:
X_train_text = [' '.join(words) for words in train_tups[0]]


vectorizer = CountVectorizer(binary=True)
X_train_vectorized = vectorizer.fit_transform(X_train_text)

print(len(vectorizer.vocabulary_))

22163


In [12]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vectorized, train_tups[1] )

test_review1 = ["the food was terrible"]
test_review2 = ["I loved my meal"]

vectorized_test_review1 = vectorizer.transform(test_review1)
vectorized_test_review2 = vectorizer.transform(test_review2)

prediction1 = model.predict(vectorized_test_review1)
prediction2 = model.predict(vectorized_test_review2)

print(prediction1)
print(prediction2)


[1]
[5]
